In [1]:
!pip install transformers

    100% |████████████████████████████████| 778kB 6.0MB/s ta 0:00:01
    100% |████████████████████████████████| 890kB 4.1MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 4.2MB/s ta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install clean-text[gpl]

    100% |████████████████████████████████| 71kB 651kB/s ta 0:00:011
    100% |████████████████████████████████| 245kB 1.8MB/s ta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ftfy
textacy 0.6.2 has requirement ftfy<5.0.0,>=4.2.0, but you'll have ftfy 5.8 which is incompatible.
  Found existing installation: ftfy 4.4.3
    Uninstalling ftfy-4.4.3:
      Successfully uninstalled ftfy-4.4.3
  Found existing installation: Unidecode 1.0.23
    Uninstalling Unidecode-1.0.23:
      Successfully uninstalled Unidecode-1.0.23
You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import os
import sys
import random
import keras
import tensorflow as tf
import json
sys.path.insert(0, '../input/pretrained-bert-including-scripts/master/bert-master')
!cp -r '../input/kerasbert/keras_bert' '/kaggle/working'
BERT_PRETRAINED_DIR = '../input/bert-base-folder'
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))

Using TensorFlow backend.


***** BERT pretrained directory: ../input/bert-base-folder *****


## Load raw model

In [4]:
from keras_bert.keras_bert.bert import get_model
from keras_bert.keras_bert.loader import load_trained_model_from_checkpoint
from keras.optimizers import Adam
adam = Adam(lr=2e-5,decay=0.01)
maxlen = 50
print('begin_build')

config_file = os.path.join('../input/bert-base-folder/bert_config.json')
checkpoint_file = os.path.join('../input/bert-base-folder/bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True,seq_len=maxlen)
model.summary(line_length=120)

Instructions for updating:
Colocations handled automatically by placer.
begin_build
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               (None, 50)                 0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             (None, 50)                 0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 50, 768), (30522,  23440896 

In [5]:
import transformers
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
save_path = 'distilbert_base_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
tokenizer.save_pretrained(save_path)

I0825 10:15:35.362718 139826089715072 file_utils.py:39] PyTorch version 1.0.1.post2 available.
I0825 10:15:37.059812 139826089715072 filelock.py:254] Lock 139822683457520 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
I0825 10:15:37.062653 139826089715072 file_utils.py:748] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp4y6es8jy


I0825 10:15:37.818794 139826089715072 file_utils.py:752] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0825 10:15:37.820354 139826089715072 file_utils.py:755] creating metadata file for /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0825 10:15:37.823983 139826089715072 filelock.py:317] Lock 139822683457520 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
I0825 10:15:37.824700 139826089715072 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /r

('distilbert_base_uncased/vocab.txt',
 'distilbert_base_uncased/special_tokens_map.json',
 'distilbert_base_uncased/added_tokens.json')

In [6]:
from tokenizers import BertWordPieceTokenizer
fast_tokenizer = BertWordPieceTokenizer('distilbert_base_uncased/vocab.txt', lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [7]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
import codecs

sequence_output  = model.layers[-6].output
pool_output = Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name = 'real_output')(sequence_output)
model3  = Model(inputs=model.input, outputs=pool_output)
model3.compile(loss='binary_crossentropy', optimizer=adam)
model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 50)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 50)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 50, 768), (3 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 50, 768)      1536        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

In [8]:
model3.load_weights('../input/bert-weights-tweets/bert_weights_tweet.h5')

In [9]:
!pip install tweepy

    100% |████████████████████████████████| 153kB 1.2MB/s ta 0:00:01
You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from cleantext import clean

In [11]:
  
class TwitterClient(object): 
    
    def __init__(self): 
        
        # keys and tokens from the Twitter Dev Console 
        consumer_key = ''
        consumer_secret = ''
        access_token = ''
        access_token_secret = ''
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        
        tweet=clean(tweet,
        fix_unicode=True,               # fix various unicode errors
        to_ascii=True,                  # transliterate to closest ASCII representation
        lower=True,                     # lowercase text
        no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_phone_numbers=True,         # replace all phone numbers with a special token
        no_numbers=True,               # replace all numbers with a special token
        no_digits=True,                # replace all digits with a special token
        no_currency_symbols=True,      # replace all currency symbols with a special token
        no_punct=True,                 # fully remove punctuation
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="<NUMBER>",
        replace_with_digit="0",
        replace_with_currency_symbol="<CUR>",
        lang="en"                       # set to 'de' for German special handling
        )
        return tweet
        
    def convert_lines(self,tweet, max_seq_length,tokenizer):
        max_seq_length -=2
        all_tokens = []

        tokens_a = tokenizer.tokenize(tweet)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)

        return np.array(all_tokens)
    
    def get_tweet_sentiment(self, tweet): 
        
        tweet2 = self.clean_tweet(tweet)
        
        token_input2 = self.convert_lines(tweet2,maxlen,tokenizer)
        
        seg_input2 = np.zeros((token_input2.shape[0],maxlen))
        mask_input2 = np.ones((token_input2.shape[0],maxlen))
        
        hehe = model3.predict([token_input2, seg_input2, mask_input2],verbose=1,batch_size=32)
        
        if hehe <= 0.5: 
            return 'no disaster'
        else: 
            return 'real disaster'
        
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['class'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 


In [16]:
api = TwitterClient() 
 
tweets = api.get_tweets(query = 'crime', count = 200) 


ptweets = [tweet for tweet in tweets if tweet['class'] == 'real disaster']  
print("Real Disaster tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 

ntweets = [tweet for tweet in tweets if tweet['class'] == 'no disaster'] 
print("No Disaster tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 


# printing first 5 positive tweets 
print("\n\n Real Disaster tweets:") 
for tweet in ptweets[:10]: 
    print(tweet['text']) 

# printing first 5 negative tweets 
print("\n\n No Disaster tweets:") 
for tweet in ntweets[:10]: 
    print(tweet['text']) 


1/1 [==============================] - 0s 151ms/step
Real Disaster tweets percentage: 28.35820895522388 %
No Disaster tweets percentage: 71.64179104477611 %


 Real Disaster tweets:
RT @Kamo_Palate: Lol I’m realizing a lot of people don’t think rape is an actual crime. They think it’s just a misunderstanding. An oops, l…
RT @MrAndyNgo: New footage shows Jacob Blake brawling with cops before being shot. The #Kenosha man, who survived, has a criminal history t…
RT @UnSubtleDesi: They called her ‘kafir’: Hindu woman Santola Devi was lynched to death by 7 Muslims in UP, her family alleges police apat…
RT @SherniIsBack07: Rhea's Connection with Drug Dealing???? 
Another Crime....!! 
But this can't be happen without any Big Name's support..…
RT @Alokmishra416: Big Breaking !!

Log soch rhe the Dal m kuch kala h.

Yaha to poori Daal hi kaali h.

Sare ke sare Police officials jo 1…
@mawson_craig @corneliusdelro @KarlyRican Your argument is that genocide is a political structure &amp; crime as 